In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings
pi = np.pi

In [2]:
#Laplace matrix
def LaplaceMatrix(N):
    A = np.zeros((N*N,N*N))
    for i in range(N):
        for j in range(N):
            A[i+j*N,i+j*N] = -4 #Diagonal
    for i in range(N-1):
        for j in range(N):            
            A[i+j*N,i+j*N+1] = 1 #Upper diagonal of the triangular submatrices
            A[i+j*N+1,i+j*N] = 1 #Lower diagonal of the triangular submatrices        
    for i in range(N):
        for j in range(N-1):
            A[i+j*N  , i+j*N + N] = 1 #Identity upper matrix
            A[i+j*N + N , i+j*N ] = 1 #Identity lower matrix        
    A = -A #We do this to be consistent with Stueben notes
    return A

def BoundaryConditions(N):
    #Boundary conditions vector
    f = np.zeros(N*N)
    L, H, h = 1, 1, 1/(N+1) 
    x, y = np.zeros(N), np.zeros(N)
    for i in range(N):
        x[i], y[i] = (i+1)*h*L, (i+1)*h*L
    #Boundary at x = 0
    fx = np.zeros(N) #f_0j
    #Boundary at y = 0
    fy = np.sin(2*pi*x/L) #fj0
    #Boundary at y = H
    fH = np.zeros(N) #fj4
    #Boundary at x = L 
    fL = np.zeros(N) #f4j
    for i in range(N):
        f[i+(N-1)*i] = -fy[i] #Bottom boundary (y=0)
        f[i+(N-1)*i+ N-1 ] = -fH[i] #Top boundary (y=H)
    for i in range(N):
        f[i] += -fx[i] #Left boundary (x=0)
        f[i+N*(N-1)] += - fL[i] #Right boundary (x=L)
    f = -f #Consistency with Stueben notes
    return f
def AnalyticSolution(x,y):
    #Here x,y are numbers
    return np.sin(2*pi*x/L)*np.sinh(2*pi*(H-y)/L)/np.sinh(2*pi*H/L)

In [3]:
class TwoGrid_SA:
    #Two grid with smoothed aggregation
    def __init__(self,A,f,NVar,eps,w):
        self.f = f #Boundary conditions
        self.NVar = NVar #Dimension of the matrix A, equal to N*N
        self.A = A
        self.Si = [] #Strong couplings
        self.Cj = [] #Aggregates
        self.nl_1 = None #Number of aggregates
        self.eps = eps #Parameter used to create the strong coupling
        self.w = w #Parameter for the Jacobi step
        self.ProlOp = None #Interpolation operator
        self.CoarseOp = None #Coarsening operator
        self.GalerkinOp = None #Galerkin operator
        self.u = np.zeros(self.NVar) #Initial guess for the solution.
        
    def get_Matrix(self):
        return self.A
    def get_Si(self):
        return self.Si
    def get_ProlOp(self):
        return self.ProlOp
    
    def StrongCouplings(self):
    #Epsilon is used to define the strong couplings
        self.Si = []
        for i in range(self.NVar):
            B = set()
            for j in range(self.NVar):
                #This is done according to Vanek et. al Computing 56 (1996)
                if abs(self.A[i,j]) >= self.eps*np.sqrt(self.A[i,i]*self.A[j,j]):
                    B.add(j)
            self.Si.append(B)
    

    def Aggregates(self): 
        R = set()
        self.Cj = []
        for i in range(self.NVar):
            R.add(i)
        j = 0
        for i in range(self.NVar):
            if self.Si[i].issubset(R):
                self.Cj.append(self.Si[i])
                j = j+1
                R = R.difference(self.Si[i])
        #If there is any i left       
        if R != set():
            for i in R:
                Values = np.zeros(j)
                count, index = 0, 0
                for k in range(j):
                    if self.Si[i].intersection(self.Cj[k]) != set():
                        #If the intersection is empty, python returns set(), not {}
                        Values[k] = abs(self.A[i,k])/np.sqrt(self.A[i,i]*self.A[k,k])
                        count += 1
                        index = k
                if count > 1:
                    maximum=0
                    for k,value in enumerate(Values):
                        if value>maximum:
                            maximum=value
                            index=k
                if count != 0:
                    self.Cj[index] = self.Cj[index].union({i})
                    R = R.difference({i})
                    #print(self.Cj,i)
        #If there is still any i left. This is part is very unlikely to happen, it might even be impossible, but
        #the paper still mentions it. In case the code prints 'here', I'll have to check if this works correctly.
        if R != set():
            print('here')
            for i in R:
                self.Cj[j].append(R.intersection(self.Si[i]))
                j = j+1
                R = R.difference(self.Cj[j])
        self.nl_1 = len(self.Cj)

    
    def ConstructOperators(self):
        self.ProlOp = np.zeros((self.NVar,self.nl_1))
        if self.NVar == self.nl_1:
            return warnings.warn("Warning, the number of variables is equal to the number of aggregates")
            
        for i in range(self.NVar):
            for j in range(self.nl_1):
                if i in self.Cj[j]:
                    self.ProlOp[i,j] = 1 #Piecewise constant interpolator
        AF = np.zeros((self.NVar,self.NVar))
        Dinv = np.zeros((self.NVar,self.NVar))
        for i in range(self.NVar):
            for j in range(self.NVar):
                if j in self.Si[i] and i != j:
                    AF[i,j] = A[i,j]
        for i in range(self.NVar):
            sumA = 0
            for k in range(self.NVar):
                if k != i:
                    sumA += A[i,k] - AF[i,k]
            AF[i,i] = A[i,i] - sumA  
            Dinv = 1/AF[i,i]
        self.ProlOp = np.dot( np.identity(self.NVar) - self.w*np.dot(Dinv,AF), self.ProlOp )
        self.CoarseOp = np.transpose(self.ProlOp)
        self.GalerkinOp = np.dot(np.dot(self.CoarseOp,self.A),self.ProlOp) 
 
    def GaussSeidel(self,nu,initial_guess):
        #f has the boundary conditions
        #nu is the number of iterations        
        self.u = initial_guess
        for k in range(nu):
            for i in range(self.NVar):
                SumA = np.dot(self.A[i,i+1:],self.u[i+1:])
                SumB = np.dot(self.A[i,:i],self.u[:i]) 
                self.u[i] = (self.f[i] - SumA - SumB) / self.A[i,i]
        return self.u

    def ExactInversion(self):
        self.u = np.dot(np.linalg.inv(self.A),self.f)
        return self.u

    def set_up_phase(self):
        self.StrongCouplings() #Construct the strong couplings
        self.Aggregates() #Form the aggregates
        self.ConstructOperators() #Construct operators 

    def TwoGridMethod(self,nu1,nu2,initial_guess):
        #I have to call the set up phase when I call the function
        #nu1 --> pre-smoothing,  nu2 --> post-smoothing
        print('Number of aggregates',len(self.Cj))
        self.u = self.GaussSeidel(nu1,initial_guess) #Pre-smoothing
        errorC = np.dot(np.dot(np.linalg.inv(self.GalerkinOp),self.CoarseOp), self.f-np.dot(self.A,self.u) )
        self.u = self.u + np.dot(self.ProlOp,errorC) #Correction 
        self.u = self.GaussSeidel(nu2,self.u) #Post-smoothing
        return self.u

In [4]:
class AMG_SA:
    #AMG with smoothed aggregation
    def __init__(self,A,NVar,maxl,eps,w):
        self.NVar = NVar #Dimension of the matrix A, equal to N*N
        self.Al = [A]
        self.eps = eps #Parameter used to create the strong coupling
        self.w = w #Parameter for the Jacobi step
        self.ProlOpl = [] #Interpolation operator
        self.maxl = maxl

    def get_Al(self):
        return self.Al
    def get_ProlOpl(self):
        return self.ProlOpl

    def set_up_phase(self):
        for i in range(self.maxl):
            if i != maxl-1:
                Nl = self.Al[i].shape[0] #Number of variables at level l
                epsilon = self.eps*(0.5)**i #epsilon = epsilon0 * 0.5^l (based on Vanek's paper)
                print('eps at level l={0}: {1}'.format(i,epsilon))
                Syst_l = TwoGrid_SA(self.Al[i],np.zeros(Nl),Nl,epsilon,self.w) #I don't actually need the second argument, but
                #the way my program is written asks me for that thing to work.
                Syst_l.StrongCouplings() #Si at the l-level
                Syst_l.Aggregates() #Build the aggregates from l-level to l+1-level
                Syst_l.ConstructOperators() #Construct operators from l-level to l+1-level
                GalerkinOp = np.dot(
                    np.dot( np.transpose(Syst_l.get_ProlOp()), Syst_l.get_Matrix() ), 
                    Syst_l.get_ProlOp() ) 
                self.Al.append(GalerkinOp) #Store A at level l+1
                self.ProlOpl.append(Syst_l.get_ProlOp()) #Store the interpolation operator from l+1 -level to l-level 
    
    def GaussSeidel(self,nu,fl,ul,level):
        #nu is the number of iterations
        Nl = len(ul)
        for k in range(nu):
            for i in range(Nl):
                SumA = np.dot(self.Al[level][i,i+1:],ul[i+1:])
                SumB = np.dot(self.Al[level][i,:i],ul[:i]) 
                ul[i] = (fl[i] - SumA - SumB) / self.Al[level][i,i]
        return ul

    def V_cycle(self,nu1,nu2,l,fl,ul):
        """
        Multi-grid with V-Cycle and aggregation
        maxl is the finest level
        nu1: pre-smoothing
        nu2: post-smoothing
        We count l starting from zero
        """    
        print('Number of aggregates at level {0}: {1}'.format(l,self.Al[l].shape[0])) 
        if l == self.maxl - 1:
            return np.dot(np.linalg.inv(self.Al[l]),fl) #At this stage we only invert the matrix that's left  
        else:    
            ul = self.GaussSeidel(nu1,fl,ul,l) #Pre-smoothing. We don't want pre smoothing for the
            #exact solution
            fl_1 = np.dot( np.transpose(self.ProlOpl[l]),
                        fl-np.dot(self.Al[l],ul) ) #restriction
            ul_1 = np.zeros(self.Al[l+1].shape[0]) #Vector for the next level. 
            ul_1 = self.V_cycle(nu1,nu2,l+1,fl_1,ul_1)  
    
        ul = ul + np.dot(self.ProlOpl[l],ul_1) #Correction
        ul = self.GaussSeidel(nu2,fl,ul,l) #Pre-smoothing  #Post-smoothing
        return ul

In [5]:
N = 50
L, H, h = 1, 1, 1/(N+1) 
x, y = np.zeros(N), np.zeros(N)
for i in range(N):
    x[i], y[i] = (i+1)*h*L, (i+1)*h*L
A = LaplaceMatrix(N)
f = BoundaryConditions(N)
print('Number of variables on the grid',N*N)

Number of variables on the grid 2500


In [6]:
maxl, eps, w = 4, 0.08, 2/3 #We need a small epsilon to form more aggregates at coarser levels
AMG = AMG_SA(A,N*N,maxl,eps,w) 
AMG.set_up_phase()

eps at level l=0: 0.08
eps at level l=1: 0.04
eps at level l=2: 0.02


In [7]:
nu1, nu2, l = 5, 5, 0 #We start at l=0
u0 = np.zeros(N*N) #Initial guess
TIME = []
nmeas = 10
for i in range(nmeas):
    start = time.time()
    uAMG = AMG.V_cycle(nu1,nu2,l,f,u0) #We are not taking the set up phase into account
    end = time.time()
    TIME.append(end-start)
TIME = np.array(TIME)
print('Execution time',np.mean(TIME),'+-',np.std(TIME)/np.sqrt(nmeas),'seconds')

Number of aggregates at level 0: 2500
Number of aggregates at level 1: 425
Number of aggregates at level 2: 52
Number of aggregates at level 3: 8
Number of aggregates at level 0: 2500
Number of aggregates at level 1: 425
Number of aggregates at level 2: 52
Number of aggregates at level 3: 8
Number of aggregates at level 0: 2500
Number of aggregates at level 1: 425
Number of aggregates at level 2: 52
Number of aggregates at level 3: 8
Number of aggregates at level 0: 2500
Number of aggregates at level 1: 425
Number of aggregates at level 2: 52
Number of aggregates at level 3: 8
Number of aggregates at level 0: 2500
Number of aggregates at level 1: 425
Number of aggregates at level 2: 52
Number of aggregates at level 3: 8
Number of aggregates at level 0: 2500
Number of aggregates at level 1: 425
Number of aggregates at level 2: 52
Number of aggregates at level 3: 8
Number of aggregates at level 0: 2500
Number of aggregates at level 1: 425
Number of aggregates at level 2: 52
Number of agg

In [8]:
eps, w = 0.2, 2/3
#The parameter epsilon is important. If it is too close to 1, it is possible to obtain a number
#of aggregates equal to the number of variables and then we will only invert the input matrix.
Sol = TwoGrid_SA(A,f,N*N,eps,w)
Sol.set_up_phase()

In [9]:
TIME = []
nmeas = 10
for i in range(nmeas):
    start = time.time()
    uFD = Sol.ExactInversion()
    end = time.time()
    TIME.append(end-start)
TIME = np.array(TIME)
print('Execution time',np.mean(TIME),'+-',np.std(TIME)/np.sqrt(nmeas),'seconds')

Execution time 0.19454576969146728 +- 0.0025653486035467203 seconds


In [10]:
u0 = np.zeros(N*N) #Initial guess
nu = 10
uGS = Sol.GaussSeidel(nu,u0)

In [11]:
nu1, nu2 = 5, 5
u0 = np.zeros(N*N)
TIME = []
nmeas = 10
for i in range(nmeas):
    start = time.time()
    uAMG_SA = Sol.TwoGridMethod(nu1,nu2,u0)
    end = time.time()
    TIME.append(end-start)
TIME = np.array(TIME)
print('Execution time',np.mean(TIME),'+-',np.std(TIME)/np.sqrt(nmeas),'seconds')

Number of aggregates 425
Number of aggregates 425
Number of aggregates 425
Number of aggregates 425
Number of aggregates 425
Number of aggregates 425
Number of aggregates 425
Number of aggregates 425
Number of aggregates 425
Number of aggregates 425
Execution time 0.09830305576324463 +- 0.0016522463855709333 seconds


In [12]:
for i in range(N):
    for j in range(N):
        string = 'u{:>0}{:>0} | Analytical {:>8} | Exact Inversion {:>8}'+\
              ' | Gauss-Seidel {:>8} | Two-Grid {:>8}' +\
              ' | AMG {:>0} levels {:>8}'
        print(string.format(i+1,j+1,
        np.round(AnalyticSolution(x[i],y[j]),5),
        np.round(uFD[i*N+j],5), 
        np.round(uGS[i*N+j],5),
        np.round(uAMG_SA[i*N+j],5),
        maxl, np.round(uAMG[i*N+j],5)
        ))  

u11 | Analytical  0.10864 | Exact Inversion  0.10866 | Gauss-Seidel  0.08808 | Two-Grid  0.10839 | AMG 4 levels  0.10821
u12 | Analytical  0.09605 | Exact Inversion  0.09608 | Gauss-Seidel  0.05951 | Two-Grid  0.09559 | AMG 4 levels  0.09523
u13 | Analytical  0.08492 | Exact Inversion  0.08496 | Gauss-Seidel  0.03803 | Two-Grid   0.0842 | AMG 4 levels  0.08367
u14 | Analytical  0.07507 | Exact Inversion  0.07512 | Gauss-Seidel  0.02309 | Two-Grid  0.07411 | AMG 4 levels  0.07341
u15 | Analytical  0.06637 | Exact Inversion  0.06642 | Gauss-Seidel  0.01337 | Two-Grid  0.06525 | AMG 4 levels   0.0644
u16 | Analytical  0.05868 | Exact Inversion  0.05873 | Gauss-Seidel  0.00742 | Two-Grid  0.05741 | AMG 4 levels  0.05641
u17 | Analytical  0.05188 | Exact Inversion  0.05193 | Gauss-Seidel  0.00396 | Two-Grid  0.05048 | AMG 4 levels  0.04936
u18 | Analytical  0.04586 | Exact Inversion  0.04592 | Gauss-Seidel  0.00204 | Two-Grid  0.04442 | AMG 4 levels  0.04319
u19 | Analytical  0.04055 | Exac